# Minimal Fake SecLink Table Setup (Timestamp Fixed)
## Apache Iceberg with Lakekeeper Catalog and MinIO Storage

## 🎯 Objective

Create a minimal Apache Iceberg table for `fake_seclink` using **PyIceberg** with **Lakekeeper** catalog and **MinIO** storage.

### **Table Schema**
```sql
CREATE TABLE fake_seclink (
    Id INT,
    TelegramCode INT,
    Source INT,
    Destination INT,
    DateIn TIMESTAMP,
    DateOut TIMESTAMP,
    Body VARCHAR(max)
);
```

### **Partitioning Strategy**
- **Primary Partition**: `month(DateIn)` - Time-based partitioning
- **Benefits**: Efficient time-based queries and data organization

### **⚠️ Important Fix**
- **Timestamp Precision**: Using microsecond precision to avoid Iceberg compatibility issues
- **PyArrow Schema**: Explicitly defining schema with `timestamp('us')`

In [1]:
from pyiceberg.catalog.rest import RestCatalog
import pandas as pd
from pyiceberg.schema import Schema
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.types import NestedField, StringType, IntegerType, TimestampType
from pyiceberg.transforms import MonthTransform
import random
from datetime import datetime, timedelta
import pyarrow as pa

# Catalog configuration
CATALOG_URL = "http://lakekeeper:8181/catalog"
WAREHOUSE = "irisa-ot"
NAMESPACE = "irisa"
TABLE_NAME = "fake_seclink"

print(f"🔧 Configuration:")
print(f"   - Catalog URL: {CATALOG_URL}")
print(f"   - Warehouse: {WAREHOUSE}")
print(f"   - Namespace: {NAMESPACE}")
print(f"   - Table: {TABLE_NAME}")

🔧 Configuration:
   - Catalog URL: http://lakekeeper:8181/catalog
   - Warehouse: irisa-ot
   - Namespace: irisa
   - Table: fake_seclink


In [2]:
# Initialize the catalog
catalog = RestCatalog(
    name="irisa_catalog",
    warehouse=WAREHOUSE,
    uri=CATALOG_URL,
    token="dummy",
)

print("✓ Catalog initialized successfully")
print(f"Available namespaces: {list(catalog.list_namespaces())}")

✓ Catalog initialized successfully
Available namespaces: []


In [3]:
# Create the irisa namespace if it doesn't exist
irisa_namespace = (NAMESPACE,)

if irisa_namespace not in catalog.list_namespaces():
    catalog.create_namespace(irisa_namespace)
    print(f"✓ Created namespace: {NAMESPACE}")
else:
    print(f"ℹ Namespace '{NAMESPACE}' already exists")

print(f"📋 Available namespaces: {list(catalog.list_namespaces())}")

✓ Created namespace: irisa
📋 Available namespaces: [('irisa',)]


In [4]:
# Define the schema for fake_seclink using PyIceberg (FIXED: using TimestampType for dates)
schema = Schema(
    NestedField(field_id=1, name="Id", field_type=IntegerType(), required=True),
    NestedField(field_id=2, name="TelegramCode", field_type=IntegerType(), required=False),
    NestedField(field_id=3, name="Source", field_type=IntegerType(), required=False),
    NestedField(field_id=4, name="Destination", field_type=IntegerType(), required=False),
    NestedField(field_id=5, name="DateIn", field_type=TimestampType(), required=False),
    NestedField(field_id=6, name="DateOut", field_type=TimestampType(), required=False),
    NestedField(field_id=7, name="Body", field_type=StringType(), required=False),
)

print("📋 Schema Defined:")
print(f"   - Total fields: {len(schema.fields)}")
for field in schema.fields:
   print(f"   - {field.name}: {field.field_type}")

📋 Schema Defined:
   - Total fields: 7
   - Id: int
   - TelegramCode: int
   - Source: int
   - Destination: int
   - DateIn: timestamp
   - DateOut: timestamp
   - Body: string


In [5]:
# Define partitioning strategy (by month of DateIn)
partition_spec = PartitionSpec(
    PartitionField(
        source_id=5,  # DateIn field ID (matches the field ID in schema definition)
        field_id=1000,
        name="DateIn_month",
        transform=MonthTransform()
    )
)

print("🔧 Partitioning Strategy:")
print(f"   - Partition by: month(DateIn)")
print(f"   - Total partition fields: {len(partition_spec.fields)}")

🔧 Partitioning Strategy:
   - Partition by: month(DateIn)
   - Total partition fields: 1


In [6]:
# Create the table
table_identifier = (NAMESPACE, TABLE_NAME)

# Check if table already exists
if table_identifier in catalog.list_tables(namespace=irisa_namespace):
    print(f"⚠ Table '{TABLE_NAME}' already exists in namespace '{NAMESPACE}'")
    print("   Dropping existing table...")
    catalog.drop_table(table_identifier)
    print("   ✓ Existing table dropped")

# Create the new table
try:
    table = catalog.create_table(
        identifier=table_identifier,
        schema=schema,
        partition_spec=partition_spec
    )
    
    print(f"✅ Table created successfully!")
    print(f"   - Table: {NAMESPACE}.{TABLE_NAME}")
    print(f"   - Location: {table.location()}")
    print(f"   - Format: {table.format_version}")
    print(f"   - Partitioning: {len(partition_spec.fields)} fields")
    
except Exception as e:
    print(f"❌ Error creating table: {str(e)}")
    raise

✅ Table created successfully!
   - Table: irisa.fake_seclink
   - Location: s3://irisa-warehouse/ot/0198503b-e3f3-7643-b7a1-e4fff6a0230e/0198503b-e46a-7530-9673-d495a364eaf0
   - Format: 2
   - Partitioning: 1 fields


In [7]:
# Generate fake data for 6 months (10,000 records)
print("🎲 Generating fake data...")

# Set up date range for 6 months
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 6, 30)
total_records = 10000

# Generate random data
data = []
for i in range(total_records):
    # Random date within the 6-month period
    random_days = random.randint(0, (end_date - start_date).days)
    date_in = start_date + timedelta(days=random_days)
    
    # Random time within the day
    random_hours = random.randint(0, 23)
    random_minutes = random.randint(0, 59)
    random_seconds = random.randint(0, 59)
    date_in = date_in.replace(hour=random_hours, minute=random_minutes, second=random_seconds)
    
    # DateOut is typically 1-60 minutes after DateIn
    random_duration = random.randint(1, 60)
    date_out = date_in + timedelta(minutes=random_duration)
    
    record = {
        "Id": i + 1,
        "TelegramCode": random.randint(1000, 9999),
        "Source": random.randint(1, 5),
        "Destination": random.randint(1, 5),
        "DateIn": date_in,  # Using datetime objects directly
        "DateOut": date_out,  # Using datetime objects directly
        "Body": f"Message body for record {i + 1} from source {random.randint(1, 5)} to destination {random.randint(1, 5)}"
    }
    data.append(record)

# Convert to DataFrame
df = pd.DataFrame(data)

# 🔧 CRITICAL FIX: Convert timestamps to microsecond precision
print("🔧 Converting timestamps to microsecond precision...")
df['DateIn'] = df['DateIn'].dt.floor('us')  # Round down to microsecond precision
df['DateOut'] = df['DateOut'].dt.floor('us')  # Round down to microsecond precision
print("✅ Timestamps converted to microsecond precision")

print(f"✅ Generated {len(df)} fake records")
print(f"📅 Date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print(f"📊 Sample data:")
print(df.head())

# Show distribution by month
monthly_distribution = df['DateIn'].dt.to_period('M').value_counts().sort_index()
print(f"\n📈 Monthly distribution:")
for month, count in monthly_distribution.items():
    print(f"   - {month}: {count} records")

🎲 Generating fake data...
🔧 Converting timestamps to microsecond precision...
✅ Timestamps converted to microsecond precision
✅ Generated 10000 fake records
📅 Date range: 2024-01-01 to 2024-06-30
📊 Sample data:
   Id  TelegramCode  Source  Destination              DateIn  \
0   1          3296       3            1 2024-03-17 11:12:34   
1   2          5351       2            4 2024-03-18 08:41:47   
2   3          1420       5            1 2024-02-09 10:46:30   
3   4          7588       5            3 2024-03-12 21:54:06   
4   5          7405       1            4 2024-05-15 15:03:25   

              DateOut                                               Body  
0 2024-03-17 11:31:34  Message body for record 1 from source 1 to des...  
1 2024-03-18 09:09:47  Message body for record 2 from source 5 to des...  
2 2024-02-09 11:00:30  Message body for record 3 from source 2 to des...  
3 2024-03-12 22:07:06  Message body for record 4 from source 4 to des...  
4 2024-05-15 15:08:25  Messag

In [8]:
# Insert data into the Iceberg table (FIXED: with explicit PyArrow schema)
print("📤 Inserting data into Iceberg table...")

try:
    # Load the table
    table = catalog.load_table(table_identifier)
    
    # 🔧 CRITICAL FIX: Convert DataFrame to PyArrow table with explicit microsecond precision schema
    # Note: Making Id field required to match the table schema
    arrow_table = pa.Table.from_pandas(df, schema=pa.schema([
        pa.field("Id", pa.int32(), nullable=False),  # Required field
        pa.field("TelegramCode", pa.int32(), nullable=True),
        pa.field("Source", pa.int32(), nullable=True),
        pa.field("Destination", pa.int32(), nullable=True),
        pa.field("DateIn", pa.timestamp('us'), nullable=True),  # Microsecond precision
        pa.field("DateOut", pa.timestamp('us'), nullable=True),  # Microsecond precision
        pa.field("Body", pa.string(), nullable=True),
    ]))
    
    print("✅ PyArrow table created with microsecond precision timestamps")
    
    # Append data to the table
    table.append(arrow_table)
    
    print(f"✅ Successfully inserted {len(df)} records into {NAMESPACE}.{TABLE_NAME}")
    
    # Simple verification
    print(f"\n🔍 Data insertion completed successfully!")
    print(f"📊 Expected records: {len(df)}")
    print(f"✅ Table: {NAMESPACE}.{TABLE_NAME} is ready for queries!")
    
except Exception as e:
    print(f"❌ Error inserting data: {str(e)}")
    raise

📤 Inserting data into Iceberg table...
✅ PyArrow table created with microsecond precision timestamps
✅ Successfully inserted 10000 records into irisa.fake_seclink

🔍 Data insertion completed successfully!
📊 Expected records: 10000
✅ Table: irisa.fake_seclink is ready for queries!


In [9]:
import duckdb

# ✅ Connect to a persistent DuckDB file
con = duckdb.connect("local.duckdb")

# ✅ Install extensions (safe even if already installed)
for ext in ["iceberg", "httpfs"]:
    try:
        con.install_extension(ext)
    except duckdb.IOException as e:
        print(f"⚠️ Could not install {ext}: {e}")

# ✅ Load extensions
for ext in ["iceberg", "httpfs"]:
    try:
        con.load_extension(ext)
    except duckdb.IOException as e:
        print(f"❌ Could not load {ext}: {e}")

# ✅ Attach the Iceberg catalog
CATALOG_URL = "http://lakekeeper:8181/catalog"
WAREHOUSE = "irisa-ot"

con.sql(f"""
    ATTACH '{WAREHOUSE}' AS irisa_datalake (
        TYPE ICEBERG,
        ENDPOINT '{CATALOG_URL}',
        TOKEN ''
    )
""")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [10]:


print("🔍 Running queries to test the data and partitioning...")

# Iceberg table reference
table_ref = "irisa_datalake.irisa.fake_seclink"

# Query 1: Total record count
print("\n📊 Query 1: Total record count")
total_count = con.sql(f"SELECT COUNT(*) FROM {table_ref}").fetchone()[0]
print(f"Total records: {total_count}")

# Query 2: Records by month (demonstrating partitioning benefits) - FIXED
print("\n📅 Query 2: Records by month (partitioning test)")
for month in range(1, 7):
    result = con.sql(f"""
        SELECT COUNT(*) FROM {table_ref}
        WHERE EXTRACT(MONTH FROM DateIn) = {month} AND EXTRACT(YEAR FROM DateIn) = 2024
    """).fetchone()[0]
    print(f"Month {month} (2024-{month:02d}): {result} records")
    
# Query 3: Top sources by record count
print("\n🏢 Query 3: Top sources by record count")
top_sources = con.sql(f"""
    SELECT Source, COUNT(*) as count
    FROM {table_ref}
    GROUP BY Source
    ORDER BY count DESC
    LIMIT 5
""").fetchall()
for source, count in top_sources:
    print(f"Source {source}: {count} records")

# Query 4: Average processing time (DateOut - DateIn)
print("\n⏱️ Query 4: Average processing time analysis")
avg_stats = con.sql(f"""
    SELECT 
        AVG(EXTRACT(EPOCH FROM DateOut - DateIn) / 60) as avg_min,
        MIN(EXTRACT(EPOCH FROM DateOut - DateIn) / 60) as min_min,
        MAX(EXTRACT(EPOCH FROM DateOut - DateIn) / 60) as max_min
    FROM {table_ref}
    WHERE DateOut IS NOT NULL AND DateIn IS NOT NULL
""").fetchone()
print(f"Average processing time: {avg_stats[0]:.2f} minutes")
print(f"Min processing time: {avg_stats[1]:.2f} minutes")
print(f"Max processing time: {avg_stats[2]:.2f} minutes")

# Query 5: Busiest hour of the day
print("\n🕐 Query 5: Busiest hour of the day")
hour_stats = con.sql(f"""
    SELECT EXTRACT(HOUR FROM DateIn) as hour, COUNT(*) as count
    FROM {table_ref}
    GROUP BY hour
    ORDER BY count DESC
    LIMIT 1
""").fetchone()
print(f"Busiest hour: {int(hour_stats[0])}:00 with {hour_stats[1]} records")

# Query 6: Sample of recent records (last 10)
print("\n📋 Query 6: Sample of recent records")
recent_records = con.sql(f"""
    SELECT Id, DateIn, Source, Destination, 
           ROUND(EXTRACT(EPOCH FROM DateOut - DateIn) / 60, 2) AS ProcessingTime
    FROM {table_ref}
    WHERE DateOut IS NOT NULL AND DateIn IS NOT NULL
    ORDER BY DateIn DESC
    LIMIT 10
""").df()
print(recent_records.to_string(index=False))

# Query 7: Performance by source system
print("\n📈 Query 7: Performance analysis by source system")
performance_stats = con.sql(f"""
    SELECT 
        Source,
        ROUND(AVG(EXTRACT(EPOCH FROM DateOut - DateIn) / 60), 2) as avg_min,
        ROUND(MIN(EXTRACT(EPOCH FROM DateOut - DateIn) / 60), 2) as min_min,
        ROUND(MAX(EXTRACT(EPOCH FROM DateOut - DateIn) / 60), 2) as max_min,
        COUNT(*) as count
    FROM {table_ref}
    WHERE DateOut IS NOT NULL AND DateIn IS NOT NULL
    GROUP BY Source
    ORDER BY count DESC
""").df()
print(performance_stats.to_string(index=False))

print("\n✅ All queries completed successfully!")
print("🎯 The month partitioning is working efficiently for time-based queries!")


🔍 Running queries to test the data and partitioning...

📊 Query 1: Total record count
Total records: 10000

📅 Query 2: Records by month (partitioning test)
Month 1 (2024-01): 1684 records
Month 2 (2024-02): 1593 records
Month 3 (2024-03): 1717 records
Month 4 (2024-04): 1612 records
Month 5 (2024-05): 1703 records
Month 6 (2024-06): 1691 records

🏢 Query 3: Top sources by record count
Source 3: 2062 records
Source 2: 2042 records
Source 4: 2003 records
Source 1: 1979 records
Source 5: 1914 records

⏱️ Query 4: Average processing time analysis
Average processing time: 30.38 minutes
Min processing time: 1.00 minutes
Max processing time: 60.00 minutes

🕐 Query 5: Busiest hour of the day
Busiest hour: 15:00 with 444 records

📋 Query 6: Sample of recent records
  Id              DateIn  Source  Destination  ProcessingTime
5951 2024-06-30 23:37:25       5            5            12.0
 244 2024-06-30 23:23:27       1            4            48.0
5383 2024-06-30 23:19:29       1            1  